In [2]:
import os

import pandas as pd
from elasticsearch import Elasticsearch, helpers
from os.path import join

es = Elasticsearch()
project_dir = join(os.getcwd(), os.pardir)
process_dir = join(project_dir, 'data', 'processed')

In [28]:
import json
from src.data._load_es import load_es

for raw_fname in os.listdir(process_dir):
    
    print(join(process_dir, raw_fname))
    with open(join(process_dir,raw_fname),) as f:
        data = json.load(f)
    
    data = [x["_source"] for x in data]
    df = pd.DataFrame(data)
    df = df.where(df.notnull(), None)
    load_es(df)

/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_33.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_7.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_8.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_29.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_20.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_15.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_41.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_16.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_5.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_14.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_23.json
/home/jaredross/personal/solve-iwmi/notebooks/../data/processed/test_data_42.json
/home/jaredross/per

In [48]:
i = 0
resp = es.search(
    index="twitter",
     body={
        "size":10000,
        "query":{
            "bool":{
                "must":{
                    "term":{
                        "is_retweet":True
                    }
                }
            }
        }
    },
    scroll='3s'
)
data = []
old_scroll_id = resp['_scroll_id']
data = data + [{"text":x["_source"]["full_text_processed"] , "id":x["_id"]} for x in resp['hits']['hits']]

    
# use a 'while' iterator to loop over document 'hits'
while len(resp['hits']['hits']):

    # make a request using the Scroll API
    resp = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '2s' # length of time to keep search context
    )

    # check if there's a new scroll ID
    if old_scroll_id != resp['_scroll_id']:
        print ("NEW SCROLL ID:", resp['_scroll_id'])

    # keep track of pass scroll _id
    old_scroll_id = resp['_scroll_id']
    i += 1
    data = data + [{"text":x["_source"]["full_text_processed"] , "id":x["_id"]} for x in resp['hits']['hits']]
        
len(data)

2020-08-13 00:11:31,775 - elasticsearch - INFO - POST http://localhost:9200/twitter/_search?scroll=3s [status:200 request:0.340s]
2020-08-13 00:11:32,356 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.284s]
2020-08-13 00:11:32,870 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.277s]
2020-08-13 00:11:33,399 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.292s]
2020-08-13 00:11:34,034 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.377s]
2020-08-13 00:11:35,241 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.313s]
2020-08-13 00:11:36,006 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.367s]
2020-08-13 00:11:36,689 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll?scrol

346054

In [49]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc["text"], [doc["id"]]) for doc in data]

In [50]:
documents[0:5]

[TaggedDocument(words='talked phone mp concerned area cyclonic storm amfan ha hit bay bengal urged make people aware storm vigilantly monitor relief work public representative people get help timely manner', tags=['1262778564564451328']),
 TaggedDocument(words='amfan storm coming country amidst corona crisis appeal congress worker west bengal odisha warn people danger help bring people safer place safe', tags=['1262778555353751552']),
 TaggedDocument(words='effect cyclone amphan ha started coastal odisha', tags=['1262778518422904832']),
 TaggedDocument(words='amid coronavirus pandemic preparing massive evacuation arrival super cyclone severe region 20 year cp', tags=['1262778489746477056']),
 TaggedDocument(words='amfan storm coming country amidst corona crisis appeal congress worker west bengal odisha warn people danger help bring people safer place safe', tags=['1262778488463020032'])]

In [51]:
model = Doc2Vec(documents, vector_size=200, window=2, min_count=1, epochs=50)

2020-08-13 00:11:56,373 - gensim.models.doc2vec - INFO - collecting all words and their counts
2020-08-13 00:11:56,374 - gensim.models.doc2vec - WARNING - Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2020-08-13 00:11:56,375 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-08-13 00:11:56,479 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 1384381 words (13336502/s), 274 word types, 10000 tags
2020-08-13 00:11:56,587 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 2809612 words (13280929/s), 314 word types, 20000 tags
2020-08-13 00:11:56,688 - gensim.models.doc2vec - INFO - PROGRESS: at example #30000, processed 4257243 words (14480132/s), 353 word types, 30000 tags
2020-08-13 00:11:57,253 - gensim.models.doc2vec - INFO - PROGRESS: at example #40000, processed 5595898 words (2372432/s), 369 word types, 40000 ta

2020-08-13 00:13:02,596 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 26.17% examples, 217131 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:03,600 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 28.51% examples, 215315 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:04,605 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 30.98% examples, 214056 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:05,619 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 33.48% examples, 213102 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:06,623 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 35.74% examples, 210266 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:07,630 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 37.90% examples, 210476 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:13:08,646 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 40.15% examples, 210305 words/s, in_qsize 6, out_qsize 0
2020-0

2020-08-13 00:13:58,415 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 61.17% examples, 217875 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:13:59,424 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 63.59% examples, 217820 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:14:00,430 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 66.03% examples, 217290 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:14:01,438 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 68.53% examples, 218617 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:14:02,447 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 70.96% examples, 219191 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:14:03,448 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 73.56% examples, 219712 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:14:04,451 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 75.94% examples, 220229 words/s, in_qsize 6, out_qsize 0
2020-0

2020-08-13 00:14:53,860 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 96.13% examples, 220331 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:14:54,867 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 98.68% examples, 220563 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:14:55,372 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:14:55,376 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:14:55,380 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:14:55,381 - gensim.models.base_any2vec - INFO - EPOCH - 3 : training on 46047183 raw words (9020289 effective words) took 40.8s, 221116 effective words/s
2020-08-13 00:14:56,398 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 2.38% examples, 220961 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:14:57,405 - gensim.models.base_an

2020-08-13 00:15:46,071 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 24.26% examples, 223636 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:15:47,078 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 26.83% examples, 222244 words/s, in_qsize 4, out_qsize 1
2020-08-13 00:15:48,079 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 29.37% examples, 222368 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:15:49,083 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 31.98% examples, 221093 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:15:50,085 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 34.24% examples, 217948 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:15:51,087 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 36.53% examples, 215872 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:15:52,092 - gensim.models.base_any2vec - INFO - EPOCH 5 - PROGRESS: at 38.69% examples, 216173 words/s, in_qsize 6, out_qsize 0
2020-0

...

2020-08-13 00:44:39,608 - gensim.models.base_any2vec - INFO - EPOCH 45 - PROGRESS: at 96.54% examples, 216220 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:44:40,617 - gensim.models.base_any2vec - INFO - EPOCH 45 - PROGRESS: at 98.97% examples, 216144 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:44:41,012 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:44:41,016 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:44:41,019 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:44:41,020 - gensim.models.base_any2vec - INFO - EPOCH - 45 : training on 46047183 raw words (9017431 effective words) took 41.6s, 216639 effective words/s
2020-08-13 00:44:42,030 - gensim.models.base_any2vec - INFO - EPOCH 46 - PROGRESS: at 2.43% examples, 228484 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:44:43,039 - gensim.models.bas

2020-08-13 00:45:30,565 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 16.07% examples, 211230 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:45:31,581 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 18.42% examples, 212255 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:45:32,583 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 20.95% examples, 215454 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:45:33,585 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 23.44% examples, 214176 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:45:34,588 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 25.88% examples, 215195 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:45:35,592 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 28.32% examples, 213887 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:45:36,599 - gensim.models.base_any2vec - INFO - EPOCH 47 - PROGRESS: at 30.65% examples, 212156 words/s, in_qsize 6, out_qsize 0

2020-08-13 00:46:25,553 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 48.26% examples, 211157 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:46:26,560 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 50.65% examples, 212734 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:46:27,571 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 53.18% examples, 213156 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:46:28,573 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 55.67% examples, 212903 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:46:29,580 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 58.05% examples, 213764 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:46:30,586 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 60.47% examples, 215485 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:46:31,589 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 62.82% examples, 214717 words/s, in_qsize 6, out_qsize 0

2020-08-13 00:47:20,111 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 74.11% examples, 207322 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:21,115 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 76.29% examples, 207868 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:22,138 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 78.62% examples, 207018 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:23,144 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 80.87% examples, 206350 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:24,146 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 83.30% examples, 206590 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:25,149 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 85.64% examples, 206143 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:47:26,151 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 88.02% examples, 206780 words/s, in_qsize 5, out_qsize 0

2020-08-13 00:48:14,588 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 98.94% examples, 205463 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:48:15,048 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:48:15,057 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:48:15,058 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:48:15,059 - gensim.models.base_any2vec - INFO - EPOCH - 50 : training on 46047183 raw words (9021346 effective words) took 43.8s, 205809 effective words/s
2020-08-13 00:48:15,060 - gensim.models.base_any2vec - INFO - training on a 2302359150 raw words (451027561 effective words) took 2123.5s, 212395 effective words/s


In [53]:
model.save(join(project_dir, 'models', 'tweet2VecJared.model'))

2020-08-13 00:48:49,239 - gensim.utils - INFO - saving Doc2Vec object under /home/jaredross/personal/solve-iwmi/notebooks/../models/tweet2VecJared.model, separately None
2020-08-13 00:48:49,241 - gensim.utils - INFO - storing np array 'vectors_docs' to /home/jaredross/personal/solve-iwmi/notebooks/../models/tweet2VecJared.model.docvecs.vectors_docs.npy
2020-08-13 00:48:50,372 - gensim.utils - INFO - saved /home/jaredross/personal/solve-iwmi/notebooks/../models/tweet2VecJared.model


In [6]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec.load(join(project_dir, 'models', 'tweet2VecJared.model'))
list(model.docvecs.doctags.keys())

['1262778564564451328',
 '1262778555353751552',
 '1262778518422904832',
 '1262778489746477056',
 '1262778488463020032',
 '1262778482473361408',
 '1262778323077218304',
 '1262778230139846656',
 '1262777938887376896',
 '1262778472088231936',
 '1262778469403865088',
 '1262757072615034880',
 '1262757024296693760',
 '1262756987726528512',
 '1262756977937010688',
 '1262778435111284736',
 '1262778433089597440',
 '1262778416157224960',
 '1262776940466851840',
 '1262778401590595584',
 '1262778388093206528',
 '1262778378932764672',
 '1262778377867603968',
 '1262778375829164032',
 '1262778362600329216',
 '1262778338822815744',
 '1262778320770478080',
 '1262771040767311872',
 '1262778320409649152',
 '1262778319289888768',
 '1262762938927964160',
 '1262762580159782912',
 '1262778305549340672',
 '1262778272250814464',
 '1262778267427385344',
 '1262778262444539904',
 '1262778236855083008',
 '1262778231230550016',
 '1262778212351905792',
 '1262768006440116224',
 '1262778201954308096',
 '12627782011656

In [7]:
len(list(model.docvecs.doctags.keys()))

346054

In [8]:
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

if 1==0:
    for i in range(0,int(len(list(model.docvecs.doctags.keys()))/10000)):
        start = (i*10000)
        end = (i*10000)+10000
        print(f'{start} to {end}')
        doc_tags = list(model.docvecs.doctags.keys())
        X = model[doc_tags[start:end]]

        tsne = TSNE()
        X_tsne = tsne.fit_transform(X)
        df = pd.DataFrame(X_tsne, index=doc_tags[start:end], columns=['x', 'y'])
        with open(join(project_dir, 'models', f'tnse_{i}.model'), 'wb') as f:
            pickle.dump(tsne, f)

        pd.np.save(join(project_dir, 'models', f'tnse_vectors_{i}.npy'), X_tsne)
        
with open(join(project_dir, 'models', f'tnse_0.model'),'rb') as f:
    tsne = pickle.load(f)
    
df = pd.np.load(join(project_dir, 'models', f'tnse_vectors_0.npy'))

df = pd.DataFrame(df,
                index=pd.Index(list(model.docvecs.doctags.keys())[0:10000]),
                columns=[u'x', u'y'])

/home/jaredross/personal/solve-iwmi/venv/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


In [9]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()
df[u'tweet_id'] = df.index
df.head()

Loading BokehJS ...

,x,y,tweet_id
1262778564564451328,-28.341555,54.042286,1262778564564451328
1262778555353751552,10.154326,58.822075,1262778555353751552
1262778518422904832,-25.044109,4.203885,1262778518422904832
1262778489746477056,-38.888901,-12.969757,1262778489746477056
1262778488463020032,9.304243,60.498081,1262778488463020032


In [10]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(df)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@tweet_id') )

# draw the words as circles on the plot
tsne_plot.circle(u'x', u'y', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);
